In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

In [2]:
data=pd.read_csv("C:\\Users\\sridhar\\Desktop\\DS2\\ML course kaggle\\6  OLS, Lasso & RF-regressor\\winequality-white.csv")

In [3]:
data.shape

(4898, 12)

In [4]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [6]:
data.dtypes

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [8]:
X=data.drop("quality", axis=1, inplace=False)
Y=data["quality"]
X_train, X_holdout, Y_train, Y_holdout = train_test_split(X, Y, test_size=0.3, random_state=42)

In [9]:
scaler=StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

Train a simple linear regression model

In [10]:
linreg=LinearRegression()

In [11]:
linreg.fit(X_train_scaled, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

 What are mean squared errors of model predictions on train and holdout sets?

In [12]:
predict_linreg=linreg.predict(X_holdout_scaled)

In [13]:
print("Holdout error:",mean_squared_error(predict_linreg, Y_holdout))

Holdout error: 0.5542927330136761


In [14]:
predict_linreg_train=linreg.predict(X_train_scaled)

In [15]:
print("Train set error:", mean_squared_error(predict_linreg_train, Y_train))

Train set error: 0.5682017769839612


Sort features by their influence on the target feature (wine quality). Beware that both large positive and large negative coefficients mean large influence on target. 

Which feature this linear regression model treats as the most influential on wine quality?


In [16]:
pd.DataFrame(linreg.coef_, X_holdout.columns, columns=['Influence']).sort_values(['Influence'], ascending=False)

,Influence
residual sugar,0.384196
alcohol,0.278218
pH,0.096886
sulphates,0.078420
free sulfur dioxide,0.077791
fixed acidity,0.044302
chlorides,0.006211
total sulfur dioxide,-0.007168
citric acid,-0.010283
volatile acidity,-0.195853


In [17]:
# "density" feature is treated as most influential by linear regression

Train a LASSO model with α=0.01(weak regularization) and scaled data.

In [18]:
lasso=Lasso(alpha=0.01, random_state=17)

In [19]:
lasso.fit(X_train_scaled, Y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

In [20]:
pd.DataFrame(lasso.coef_, X_holdout.columns, columns=['Influence_lasso']).sort_values(['Influence_lasso'], ascending=False)

,Influence_lasso
alcohol,0.386255
residual sugar,0.188975
free sulfur dioxide,0.069551
sulphates,0.052095
pH,0.041171
total sulfur dioxide,-0.000000
chlorides,-0.000437
citric acid,-0.003847
fixed acidity,-0.014010
density,-0.129207


Which feature is the least informative in predicting wine quality, according to this LASSO model?


In [21]:
# "free sulfur dioxide" is least informative feature according to lasso model because it weights less among all feature-coefficients

Train LassoCV with random_state=17 to choose the best value of α in 5-fold cross-validation.

In [22]:
lassocv=LassoCV(alphas = np.logspace(-6, 2, 200), random_state=17, cv=5)

In [23]:
lassocv.fit(X_train_scaled, Y_train)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [24]:
lassocv.alpha_

0.005478901179593945

Which feature is the least informative in predicting wine quality, according to the tuned LASSO model?

In [25]:
pd.DataFrame(lassocv.coef_, X_holdout.columns, columns=['Influence_lassocv']).sort_values(['Influence_lassocv'], ascending=False)

,Influence_lassocv
alcohol,0.348724
residual sugar,0.260146
free sulfur dioxide,0.071830
sulphates,0.062554
pH,0.058950
fixed acidity,-0.000000
chlorides,-0.000000
total sulfur dioxide,-0.000762
citric acid,-0.005571
volatile acidity,-0.196070


In [26]:
# "fixed acidity" and "chlorides" features are least informative according to lassocv model

What are mean squared errors of tuned LASSO predictions on train and holdout sets?

In [27]:
predict_lassocv=lassocv.predict(X_holdout_scaled)

In [28]:
print("HOldout error of lassocv: ", mean_squared_error(predict_lassocv, Y_holdout))

HOldout error of lassocv:  0.5563658332789942


In [29]:
predict_lassocv_train=lassocv.predict(X_train_scaled)

In [30]:
print("Training error of lassocv: ", mean_squared_error(predict_lassocv_train, Y_train))

Training error of lassocv:  0.5697590575338743


Train a Random Forest with out-of-the-box parameters, setting only random_state to be 17.

In [31]:
forest = RandomForestRegressor(random_state=17)

In [32]:
forest.fit(X_train_scaled, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=17, verbose=0, warm_start=False)

What are mean squared errors of RF model on the training set, in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?

In [33]:
predict_forest=forest.predict(X_holdout_scaled)

In [34]:
print("Holdout error of Random forest model: ", mean_squared_error(predict_forest, Y_holdout))

Holdout error of Random forest model:  0.40282312925170066


In [35]:
predict_forest_training=forest.predict(X_train_scaled)

In [36]:
print("Training error of Random forest model: ", mean_squared_error(predict_forest_training, Y_train))

Training error of Random forest model:  0.07960618436406067


In [37]:
cross_val_score(X=X_train_scaled, y=Y_train, cv=5, estimator=forest,scoring='neg_mean_squared_error')

array([-0.39138484, -0.44721574, -0.46779883, -0.47391241, -0.51010219])

Tune the max_features and max_depth hyperparameters with GridSearchCV and again check mean cross-validation MSE and MSE on holdout set.

In [38]:
parameters={'max_depth':list(range(10, 25)),'max_features':[2,3,4,5,6,7,8,9]}

In [39]:
grid=GridSearchCV(RandomForestRegressor(random_state=17), parameters, cv=5)

In [40]:
grid.fit(X_train_scaled, Y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 'max_features': [2, 3, 4, 5, 6, 7, 8, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [41]:
grid.best_params_

{'max_depth': 15, 'max_features': 3}

In [42]:
forest_tuned=RandomForestRegressor(max_depth=15, max_features=3, random_state=17)

In [43]:
forest_tuned.fit(X_train_scaled, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features=3, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [44]:
predict_forest_tuned=forest_tuned.predict(X_holdout_scaled)

In [45]:
print("HOldout error of forest_tuned: ", mean_squared_error(predict_forest_tuned, Y_holdout))

HOldout error of forest_tuned:  0.3952659397233657


In [46]:
cross_val_score(X=X_train_scaled, y=Y_train, cv=5, estimator=forest_tuned,scoring='neg_mean_squared_error')

array([-0.39830828, -0.43758338, -0.46295186, -0.46317414, -0.44985613])

In [47]:
grid.best_score_

0.4441386024190344

What is the most important feature, according to the Random Forest model?

In [48]:
rf_importance = pd.DataFrame(forest_tuned.feature_importances_, X_train.columns ,columns=['Feature imp'])

In [49]:
rf_importance.sort_values(by=['Feature imp'],ascending=False)

,Feature imp
alcohol,0.183095
density,0.110157
volatile acidity,0.109632
free sulfur dioxide,0.107889
chlorides,0.091822
total sulfur dioxide,0.070025
residual sugar,0.069046
fixed acidity,0.067056
citric acid,0.065189
pH,0.064417


In [50]:
# "alcohol" feature is the most important according to random forest model